# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools
theme(:dracula)

First, we define the Honeycomb structure, and some useful functions on it:

In [ ]:
mutable struct HoneycombLattice
    shape::String
    a::Real
    verts::Vector{Tuple{Int64, Tuple}}
    bonds::Vector{Tuple}
    ψ::Vector{Array{Float64}}

    function HoneycombLattice(shape::String, a::Real, shift::Tuple, n::Integer, bc::String)
        @assert shape ∈ ("pyramid", "flower", "bricks") "Invalid lattice shape"
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        @assert n > 0 "Lattice order `n` must be positive"
        @assert bc ∈ ("periodic", "open") "Invalid lattice boundary condition"

        # define verts location based on lattice shape
        if shape == "pyramid"
            dx = a * sin(π/6)
            dy = a * cos(π/6)
            
            locs = [(0.0, 0.0)]
            for (i,x) in enumerate(range(start=dx, length=n, step=a+dx))
                for y in range(-dy*i, stop=dy*i, length=i+1)
                    append!(locs, [(-x,y), (-x-a,y)])
                end
            end
        
            for (j, y) in enumerate(range(start=-dy*(n-1), stop=dy*(n-1), length=n))
                append!(locs, [(-n*(dx+a)-dx, y)])
            end
        elseif shape == "bricks"
            @assert (n % 2 == 0) || (n == 1) "Number of bricks must be 1 or an even number"
            if n == 1
                locs = [(0.0, 0.0), (0.0, a), (2*a, 0.0), (a, a), (a, 0.0), (2*a, a)]
            else
                locs = [(0.0, 0.0), (0.0, a), (a, 2*a), (a, a)]
                for i in 1:n
                    if i%2==0
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,2*a),(2*a,2*a),(2*a,a)]])
                    else
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,0),(2*a,0),(2*a,a)]])
                    end
                end
            end            
        end
        
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # detect bonds
        bonds = detectbonds(verts, a, shape)

        # fix boundary conditions
        @assert bc == "open" "Non-open boundary conditions not implemented yet"
        
        # fill in tensors
        A = AKLT_PEPS()
        missing = detectmissing(verts, bonds)
        ψ = []
        for (vert, miss) in zip(verts, missing)
            (ind, loc) = vert
            if length(miss) == 0
                A_cropped = A
            elseif length(miss) == 1
                A_cropped = reshape(A[:,:,:,1], (size(A[:,:,:,1])...,1)) #A[:,:,:,1]
            elseif length(miss) == 2
                A_cropped = reshape(A[:,:,1,1], (size(A[:,:,1,1])...,1,1)) #A[:,:,1,1]
            elseif length(miss) == 3
                A_cropped = reshape(A[:,1,1,1], (size(A[:,1,1,1])...,1,1,1)) #A[:,1,1,1]
            else
                error("Invalid number of missing bonds from vertex ", ind)
            end
            push!(ψ, A_cropped)
        end
        
        # create structure
        new(shape, a, verts, bonds, ψ)
    end
end

In [ ]:
function detectbonds(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, a::Real, shape::String=nothing)
    bonds = []
    bond_count = ones(Int64, length(verts))
    for vert in verts
        (ind, loc) = vert
        for (ind_j, loc_j) in nearestneighboors(verts[ind:end], vert, a)
            if shape =="bricks" && (ind_j == ind + 1 && ind % 3 == 1 && ind != 1)
                continue
            end
            push!(bonds, ((ind, bond_count[ind]), (ind_j, bond_count[ind_j])))
            bond_count[ind] += 1
            bond_count[ind_j] += 1
        end
    end
    return bonds
end

In [ ]:
function countneighbors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, bonds::Vector{Any})
    c = 0
    vert_bonds = []
    (ind, loc) = vert
    for bond in bonds
        if ind ∈ bond
            c += 1
            push!(vert_bonds, bond)
        end
    end
    return c, vert_bonds
end

In [ ]:
function nearestneighboors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, a::Real)
    neighbors = []
    (ind, loc) = vert
    for (ind_i, loc_i) in verts
        d = sum((loc .- loc_i).^2)
        if abs(d - a) < 1e-1*a
            push!(neighbors, (ind_i, loc_i))
        end
    end
    return neighbors
end

In [ ]:
function detectmissing(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, bonds::Vector{Any})
    missing = [[1,2,3] for i in 1:length(verts)]
    for bond in bonds
        for (vert_ind, bond_ind) in bond
            deleteat!(missing[vert_ind], findall(x->x==bond_ind,missing[vert_ind]))
        end
    end
    return missing
end

In [ ]:
function showlattice(fig::Plots.Plot, lat::HoneycombLattice, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]
    for bond in lat.bonds
        inds = [b[1] for b in bond]
        dx = [lat.verts[i][2][1] for i in inds]
        dy = [lat.verts[i][2][2] for i in inds]
        plot!(fig, dx, dy, label="", color="cyan")
    end
    scatter!(fig, x, y, aspect_ratio=1, label="", color="dark red")

    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

The tensors are defined using the following function:

In [ ]:
function AKLT_PEPS()
    Θ = zeros(4,2,2,2)
    Σ = zeros(2,2)
    A = zeros(4,2,2,2)
    
    # define Σ the singlet matrix
    Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);
    
    # define Θ the projection operator
    for a in 1:2
        for b in 1:2
            for c in 1:2
                Θ[1,a,b,c] = (a+b+c == 3)
                Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
                Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
                Θ[4,a,b,c] = (a+b+c == 6)
            end
        end
    end
    
    # define A the total tensor
    for σ in 1:4
        for b in 1:2
            A[σ,:,b,:] = Θ[σ,:,b,:]*Σ
        end
    end
    return A
end

For example, here are two possible lattices:

In [ ]:
lat = HoneycombLattice("bricks", 1, (5.0,1.0), 4, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

In [ ]:
lat = HoneycombLattice("pyramid", 1, (5.0,1.0), 2, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

Next, in order to calculate local expectation values of a specific site, we need to orthogonalize the PEPS onto the site. This is done by contracting bonds of sites and then split them again using SVD, and is the reason for the more complicated bond data-structure in the `HoneycombLattice` struct. We start by writing a function to normalize the PEPS:

In [ ]:
function bondtensor(lat::HoneycombLattice, ind_i::Int64, ind_j::Int64)
    @assert ind_i >= 1 && ind_i <= length(lat.ψ) "Bond 1 out of range"
    @assert ind_j >= 1 && ind_j <= length(lat.ψ) "Bond 2 out of range"
    @assert ind_i != ind_j "Bond indices cannot be equal"

    # fond bond in list of bonds
    bond = findbond(lat, ind_i, ind_j)
    
    # contract together sites ind_i and ind_j according to their bond indices
    if bond[1][2] == 1 && bond[2][2] == 1
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,l,a,b] * lat.ψ[ind_j][σj,l,c,d]
    elseif bond[1][2] == 2 && bond[2][2] == 1
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,a,l,b] * lat.ψ[ind_j][σj,l,c,d]
    elseif bond[1][2] == 3 && bond[2][2] == 1
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,a,b,l] * lat.ψ[ind_j][σj,l,c,d]
    elseif bond[1][2] == 1 && bond[2][2] == 2
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,l,a,b] * lat.ψ[ind_j][σj,c,l,d]
    elseif bond[1][2] == 2 && bond[2][2] == 2
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,a,l,b] * lat.ψ[ind_j][σj,c,l,d]
    elseif bond[1][2] == 3 && bond[2][2] == 2
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,a,b,l] * lat.ψ[ind_j][σj,c,l,d]
    elseif bond[1][2] == 1 && bond[2][2] == 3
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,l,a,b] * lat.ψ[ind_j][σj,c,d,l]
    elseif bond[1][2] == 2 && bond[2][2] == 3
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,a,l,b] * lat.ψ[ind_j][σj,c,d,l]
    elseif bond[1][2] == 3 && bond[2][2] == 3
        @tensor ψ_bond[a,b,σi,σj,c,d] := lat.ψ[ind_i][σi,a,b,l] * lat.ψ[ind_j][σj,c,d,l]
    end

    return ψ_bond
end

In [ ]:
function SVD_bondtensor(lat::HoneycombLattice, ind_i::Int64, ind_j::Int64, ortho_ind::Int64, maxM::Int64=-1, renorm::Bool=false)
    # verify orthogonality center
    @assert (ortho_ind == ind_i) || (ortho_ind == ind_j) "Orthogonality index must be either i or j"
    
    # get bond tensor of the i-j bond
    ψ_bond = bondtensor(lat, ind_i, ind_j)
    
    # Reshape the bond tensor into a matrix for SVD
    M0 = prod(size(ψ_bond)[1:3])
    M2 = prod(size(ψ_bond)[4:6])
    ψ_bond_matrix = reshape(ψ_bond, (M0, M2))

    # Perform SVD on the reshaped bond tensor matrix
    U, S, V = svd(ψ_bond_matrix)
    
    # Optionally renormalize the singular values
    if renorm
        S /= norm(S)
    end

    # Find the last non-zero singular value
    M1 = findlast(S .> 1e-14)
    
    # Retain only relevant parts of U, S, and V
    U, S, V = U[:, 1:M1], S[1:M1], V[:, 1:M1]

    # If 0 < maxM < M1 - truncation, perform truncation
    trunc = 0.
    if maxM > 0 && M1 > maxM
        trunc = sum(S[maxM:end].^2)
        U, S, V = U[:, 1:maxM], S[1:maxM], V[:, 1:maxM]
        S /= sqrt(1. - trunc)
        M1 = maxM
    end

    # calculate the shape of the final tensors based on the bond dimensions
    bond = findbond(lat, ind_i, ind_j)
    if bond[1][2] == 1 && bond[2][2] == 1
        size_i = (4, M1, size(ψ_bond)[1], size(ψ_bond)[2])
        size_j = (4, M1, size(ψ_bond)[5], size(ψ_bond)[6])
    elseif bond[1][2] == 2 && bond[2][2] == 1
        size_i = (4, size(ψ_bond)[1], M1, size(ψ_bond)[2])
        size_j = (4, M1, size(ψ_bond)[5], size(ψ_bond)[6])
    elseif bond[1][2] == 3 && bond[2][2] == 1
        size_i = (4, size(ψ_bond)[1], size(ψ_bond)[2], M1)
        size_j = (4, M1, size(ψ_bond)[5], size(ψ_bond)[6])
    elseif bond[1][2] == 1 && bond[2][2] == 2
        size_i = (4, M1, size(ψ_bond)[1], size(ψ_bond)[2])
        size_j = (4, size(ψ_bond)[5], M1, size(ψ_bond)[6])
    elseif bond[1][2] == 2 && bond[2][2] == 2
        size_i = (4, size(ψ_bond)[1], M1, size(ψ_bond)[2])
        size_j = (4, size(ψ_bond)[5], M1, size(ψ_bond)[6])
    elseif bond[1][2] == 3 && bond[2][2] == 2
        size_i = (4, size(ψ_bond)[1], size(ψ_bond)[2], M1)
        size_j = (4, size(ψ_bond)[5], M1, size(ψ_bond)[6])
    elseif bond[1][2] == 1 && bond[2][2] == 3
        size_i = (4, M1, size(ψ_bond)[1], size(ψ_bond)[2])
        size_j = (4, size(ψ_bond)[5], size(ψ_bond)[6], M1)
    elseif bond[1][2] == 2 && bond[2][2] == 3
        size_i = (4, size(ψ_bond)[1], M1, size(ψ_bond)[2])
        size_j = (4, size(ψ_bond)[5], size(ψ_bond)[6], M1)
    elseif bond[1][2] == 3 && bond[2][2] == 3
        size_i = (4, size(ψ_bond)[1], size(ψ_bond)[2], M1)
        size_j = (4, size(ψ_bond)[5], size(ψ_bond)[6], M1)
    end
    
    # Update the PEPS tensors based on the SVD results
    if ortho_ind == ind_j
        lat.ψ[ind_i] = reshape(U, size_i)
        lat.ψ[ind_j] = reshape(Diagonal(S) * V', size_j)
    elseif ortho_ind == ind_i
        lat.ψ[ind_i] = reshape(U * Diagonal(S), size_i)
        lat.ψ[ind_j] = reshape(V', size_j)
    end
    
    return lat.ψ, trunc
end

In [ ]:
function findbond(lat::HoneycombLattice, ind_i::Int64, ind_j::Int64)
    if ind_i > ind_j
        (ind_i, ind_j) = (ind_j, ind_i)
    end
    
    found = false
    bond = []
    for b in lat.bonds
        ind_bond = (b[1][1], b[2][1])
        if (ind_i, ind_j) == ind_bond
            found = true
            bond = b
            break
        end
    end
    @assert found==true "Bond does not appear in lattice"
    return bond
end

In [ ]:
function sortbonds(lat::HoneycombLattice)
	bonds = []
	for bond in lat.bonds
		ind_i = bond[1][1]
		ind_j = bond[2][1]

		vert_i = lat.verts[ind_i]
		vert_j = lat.verts[ind_j]
		
		loc_i = vert_i[2]
		loc_j = vert_j[2]

		x_i = loc_i[1]
		x_j = loc_j[1]

		x_min = min(x_i, x_j)
		push!(bonds, (x_min, bond))
	end
	perm = sortperm([x for (x, bond) in bonds], rev=false)
	bonds = lat.bonds[perm]
	return bonds
end

In [ ]:
function find_topright(lat::HoneycombLattice)
	bond = sortbonds(lat)[end]
	inds = [b[1] for b in bond]
	verts = lat.verts[inds]
	xs = [v[2][1] for v in verts]
	if xs[1] > xs[2]
		return inds[1]
	elseif xs[1] < xs[2]
		return inds[2]
	else
		ys = [v[2][2] for v in verts]
		if ys[1] > ys[2]
			return inds[1]
		elseif ys[1] < ys[2]
			return inds[2]
		else
			error("Vertices have identical location")
		end
	end
end

In [ ]:
function normalize_PEPS(lat::HoneycombLattice)
	bonds = sortbonds(lat)
    ψ = []
    for bond in bonds
        ind_i, b_i = bond[1]
        ind_j, b_j = bond[2]
        _, l_i = lat.verts[ind_i]
        _, l_j = lat.verts[ind_j]
        if l_i[1] < l_j[1]
            ψ, _ = SVD_bondtensor(lat, ind_i, ind_j, ind_j, -1, true)
            # println("Moved orthogonality center from ", ind_i, " to ", ind_j)
        elseif l_i[1] > l_j[1]
            ψ, _ = SVD_bondtensor(lat, ind_i, ind_j, ind_i, -1, true)
            # println("Moved orthogonality center from ", ind_j, " to ", ind_i)
        else
            if l_i[2] < l_j[2]
            ψ, _ = SVD_bondtensor(lat, ind_i, ind_j, ind_j, -1, true)
                # println("Moved orthogonality center from ", ind_i, " to ", ind_j)
            elseif l_i[2] > l_j[2]
            ψ, _ = SVD_bondtensor(lat, ind_i, ind_j, ind_i, -1, true)
                # println("Moved orthogonality center from ", ind_j, " to ", ind_i)
            end
        end
    end
    return ψ
end

and now we can use those functions to calculate some expextation values:

In [ ]:
function expect_topright(lat::HoneycombLattice, Op)
    # normalize and orthogonalize lat to top-right site
    lat.ψ = normalize_PEPS(lat)
    
    # extract the matrix Ai from the PEPS at the top-right site
    topright = find_topright(lat)
    Ai = lat.ψ[topright]

    # Contract Ai with Op to obtain Ai_Sop tensor
    @tensor Ai_Sop[σ,a,b,c] := Ai[σp,a,b,c] * Op[σp, σ]
    
    # Calculate the expectation value
    @tensor val = Ai_Sop[σ,a,b,c] * Ai[σ,a,b,c]
    return val
end

In [ ]:
Sz = diagm([3,1,-1,3]./2)

lat = HoneycombLattice("pyramid", 1, (5.0,1.0), 1, "open")
for i in 1:10
    lat.ψ = normalize_PEPS(lat)
    @show expect_topright(lat, Sz)
end

And as expected, nothing works. Nice!